# Preprocessing parameters

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rxavier/poniard/blob/master/examples/03._preprocessing_parameters.ipynb)

This notebook outlines preprocessing parameters for Poniard estimators.

If you don't have it installed, please install from PyPI.

In [1]:
# %pip install poniard

Poniard tries to apply minimal preprocessing to data. In general, it just tries to make sure that models fit correctly without introducing signifcant transformation overhead. In particular, there is no anomaly detection, dimensionality reduction, clustering, resampling, feature creation from polynomial interactions, feature selection, etc.

This is so the user always knows what's going on.

However, the default options may not be suitable for your data or objectives, so these can be set during initialization or modified afterwards.

## Basics

The list of default transformations is:
* Missing data imputation.
* Z-score scaling for numeric variables.
* One-hot encoding for low cardinality categorical variables.
* Target encoding for the remaining categorical variables. This is a custom transformer based on Micci-Barreca, 2001, with implementation heavily based on [Dirty Cat](https://github.com/dirty-cat/dirty_cat/blob/master/dirty_cat/target_encoder.py).
* Datetime encoding for datetime variables. This also uses a custom transformer that extracts multiple datetime levels.
* Zero-variance feature elimination.

This includes some type inference logic that decides whether a given feature is either numeric, categorical high cardinality, categorical low cardinality or datetime.

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from poniard import PoniardRegressor

X, y = load_diabetes(return_X_y=True, as_frame=True)
pnd = PoniardRegressor(estimators=[Ridge()])
pnd.setup(X, y)
pnd.preprocessor_

Target info
-----------
Type: continuous
Shape: (442,)
Unique values: 214

Main metric
-----------
neg_mean_squared_error

Thresholds
----------
Minimum unique values to consider a feature numeric: 44
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,age,,sex,
1,bmi,,,
2,bp,,,
3,s1,,,
4,s2,,,
5,s3,,,
6,s4,,,
7,s5,,,
8,s6,,,


Pipeline(steps=[('type_preprocessor',
                 ColumnTransformer(transformers=[('numeric_preprocessor',
                                                  Pipeline(steps=[('numeric_imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'bp', 's1',
                                                   's2', 's3', 's4', 's5',
                                                   's6']),
                                                 ('categorical_low_preprocessor',
                                                  Pipeline(steps=[('categorical_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['sex'])])),
                ('remove_invariant', VarianceThreshold())])

`preprocess=False` disables preprocessing altogether, while `custom_preprocessor` accepts a scikit-learn transformer (or pipeline/column transformer) that replaces the default Poniard transformation pipeline.

Logically, there is no type inference involved when these options are used and full control is given to the user.

In the following example, we use `TfidfVectorizer()` to process the 20 News Groups dataset.

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression
from poniard import PoniardClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

X, y = fetch_20newsgroups(return_X_y=True, remove=("headers", "footers", "quotes"),
                          categories=("sci.crypt", "sci.electronics", "sci.med"))
pnd = PoniardClassifier(estimators=[LogisticRegression()], custom_preprocessor=TfidfVectorizer())
pnd.setup(X, y)
pnd.preprocessor_

Target info
-----------
Type: multiclass
Shape: (1780,)
Unique values: 3

Main metric
-----------
roc_auc_ovr



TfidfVectorizer()

In [4]:
pnd.fit()
pnd.show_results()

Completed: 100%|██████████| 2/2 [00:14<00:00,  7.12s/it]         


,test_roc_auc_ovr,train_roc_auc_ovr,test_accuracy,train_accuracy,test_precision_macro,train_precision_macro,test_recall_macro,train_recall_macro,test_f1_macro,train_f1_macro,fit_time,score_time
LogisticRegression,0.976337,0.998684,0.888202,0.977809,0.896170,0.978822,0.888335,0.977844,0.888822,0.977939,0.843882,0.158378
DummyClassifier,0.500000,0.500000,0.334270,0.334270,0.111423,0.111423,0.333333,0.333333,0.167018,0.167018,0.407044,0.160016


Type inference is governed by the input data types and two thresholds included in the estimator constructor.

Number features (as defined by `numpy`) with unique values greater than `numeric_threshold` will be treated as numeric, with the remainder being treated as non-numeric. If this parameter is a float, the actual threshold is `numeric_threshold * samples`.

Non-numeric features (either because they are number features below `numeric_threshold` or they are non-number features like strings) with unique values greater than `cardinality_threshold` will be considered high cardinality. Likewise, in the case of a float value, the threshold is `cardinality_threshold * samples`.

Defaults are set at reasonable limits, but do pay attention to the output of `setup()` as it might expose misclassified features. In that scenario there's three options: initialize the estimator with different thresholds that better acommodate the dataset, use a `custom_preprocessor` that applies appropiate transformations to different sets of features, or use the `reassign_types()` method to explicitly assign features to the three categories.

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.impute import KNNImputer
from poniard import PoniardRegressor

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
reg = PoniardRegressor(numeric_imputer=KNNImputer(), scaler="minmax")
reg.setup(X, y)
reg.preprocessor_

Target info
-----------
Type: continuous
Shape: (20640,)
Unique values: 3842

Main metric
-----------
neg_mean_squared_error

Thresholds
----------
Minimum unique values to consider a feature numeric: 2064
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,MedInc,HouseAge,,
1,AveRooms,Latitude,,
2,AveBedrms,Longitude,,
3,Population,,,
4,AveOccup,,,


Pipeline(steps=[('type_preprocessor',
                 ColumnTransformer(transformers=[('numeric_preprocessor',
                                                  Pipeline(steps=[('numeric_imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['MedInc', 'AveRooms',
                                                   'AveBedrms', 'Population',
                                                   'AveOccup']),
                                                 ('categorical_high_preprocessor',
                                                  Pipeline(steps=[('categorical_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('high_cardinality_encoder',
                                                                   TargetEncoder(handle_unknown='ignore',
                                                                                 task='regression'))]),
                                                  ['HouseAge', 'Latitude',
                                                   'Longitude'])])),
                ('remove_invariant', VarianceThreshold())])

In [6]:
reg.reassign_types(numeric=["AveRooms", "AveBedrms", "Population", "AveOccup", "HouseAge", "Latitude", "Longitude"])

Assigned feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,AveRooms,,,
1,AveBedrms,,,
2,Population,,,
3,AveOccup,,,
4,HouseAge,,,
5,Latitude,,,
6,Longitude,,,


PoniardRegressor(estimators=None, metrics=None,
    preprocess=True, scaler=minmax, numeric_imputer=KNNImputer(),
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

## Imputation, numeric scaling and categorical encoding

Different options for imputation, scaling and encoding are available on initialization. As a rule, `scaler`, `numeric_imputer` and `high_cardinality_encoder` accept appropiate scikit-learn preprocessors, e.g. `RobustScaler()` and `KNNImputer()` in the first two cases, and something like `OrdinalEncoder()` in the latter, which will be used as is.

Also, they accept strings that represent scikit-learn preprocessors. The scaler can be "standard", "minmax" or "robust", the imputer can be "simple" or "iterative", and the encoder can be "target" or "ordinal".

We are deliberately not providing options for the categorical imputer (a `SimpleImputer(strategy="most_frequent")` is used) or the low cardinality categorical encoder (always `OneHotEncoder()`). While this is not set in stone, we feel that these are less debatable.

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.impute import KNNImputer
from poniard import PoniardRegressor

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
reg = PoniardRegressor(numeric_imputer=KNNImputer(), scaler="minmax")
reg.setup(X, y)
reg.reassign_types(numeric=["AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"],
                   categorical_high=["HouseAge"])
reg.preprocessor_

Target info
-----------
Type: continuous
Shape: (20640,)
Unique values: 3842

Main metric
-----------
neg_mean_squared_error

Thresholds
----------
Minimum unique values to consider a feature numeric: 2064
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,MedInc,HouseAge,,
1,AveRooms,Latitude,,
2,AveBedrms,Longitude,,
3,Population,,,
4,AveOccup,,,




Assigned feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,AveRooms,HouseAge,,
1,AveBedrms,,,
2,Population,,,
3,AveOccup,,,
4,Latitude,,,
5,Longitude,,,


Pipeline(steps=[('type_preprocessor',
                 ColumnTransformer(transformers=[('numeric_preprocessor',
                                                  Pipeline(steps=[('numeric_imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['AveRooms', 'AveBedrms',
                                                   'Population', 'AveOccup',
                                                   'Latitude', 'Longitude']),
                                                 ('categorical_high_preprocessor',
                                                  Pipeline(steps=[('categorical_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('high_cardinality_encoder',
                                                                   TargetEncoder(handle_unknown='ignore',
                                                                                 task='regression'))]),
                                                  ['HouseAge'])])),
                ('remove_invariant', VarianceThreshold())])

In [8]:
reg.remove_estimators(["LinearSVR", "RandomForestRegressor", "ElasticNet"])
reg.fit()
reg.show_results()

Completed: 100%|██████████| 6/6 [00:20<00:00,  3.48s/it]                    


,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_mean_absolute_percentage_error,train_neg_mean_absolute_percentage_error,test_neg_median_absolute_error,train_neg_median_absolute_error,test_r2,train_r2,fit_time,score_time
XGBRegressor,-0.237771,-9.412031e-02,-0.185607,-1.233918e-01,-0.216376,-0.155068,0.821449,0.929319,1.141237,0.012350
HistGradientBoostingRegressor,-0.239658,-1.821939e-01,-0.190946,-1.707084e-01,-0.226846,-0.206955,0.820020,0.863170,2.122466,0.024681
KNeighborsRegressor,-0.403042,-2.622197e-01,-0.233540,-1.882364e-01,-0.284160,-0.228000,0.697173,0.803084,0.063917,0.077780
DecisionTreeRegressor,-0.465499,-8.310265e-32,-0.233377,-7.419690e-18,-0.245401,0.000000,0.650196,1.000000,0.140646,0.008378
LinearRegression,-0.791995,-7.848231e-01,-0.424454,-4.235226e-01,-0.528726,-0.527934,0.405098,0.410588,0.091159,0.015789
DummyRegressor,-1.331610,-1.331544e+00,-0.621102,-6.210980e-01,-0.762638,-0.761340,-0.000124,0.000000,0.041799,0.006458


Preprocessing steps can be added to an existing preprocessor in any position.

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression

reg.add_preprocessing_step(("feature_selection", SelectKBest(f_regression, k=5)), position="end")
reg.preprocessor_

Pipeline(steps=[('type_preprocessor',
                 ColumnTransformer(transformers=[('numeric_preprocessor',
                                                  Pipeline(steps=[('numeric_imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['AveRooms', 'AveBedrms',
                                                   'Population', 'AveOccup',
                                                   'Latitude', 'Longitude']),
                                                 ('categorical_high_preprocessor',
                                                  Pipeline(steps=[('categorical_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('high_cardinality_encoder',
                                                                   TargetEncoder(handle_unknown='ignore',
                                                                                 task='regression'))]),
                                                  ['HouseAge'])])),
                ('remove_invariant', VarianceThreshold()),
                ('feature_selection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x17e7b3f70>))])